In [2]:
import random
import pickle
import re
import pandas as pd

In [87]:
wordnet = {}
with open("wordnet.pickle", "rb") as f:
    wordnet = pickle.load(f)


# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
    parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

    return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    if len(new_words) != 0:
        sentence = ' '.join(new_words)
        new_words = sentence.split(" ")

    else:
        new_words = ""

    return new_words


def get_synonyms(word):
    synomyms = []

    try:
        for syn in wordnet[word]:
            for s in syn:
                synomyms.append(s)
    except:
        pass

    return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
    new_words = words.copy()
    for _ in range(n):
        add_word(new_words)
    
    return new_words


def add_word(new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
        if len(new_words) >= 1:
            random_word = new_words[random.randint(0, len(new_words)-1)]
            synonyms = get_synonyms(random_word)
            counter += 1
        else:
            random_word = ""

        if counter >= 10:
            return
        
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
    sentence = get_only_hangul(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word is not ""]
    num_words = len(words)

    augmented_sentences = []
    num_new_per_technique = int(num_aug/4) + 1

    n_sr = max(1, int(alpha_sr*num_words))
    n_ri = max(1, int(alpha_ri*num_words))
    n_rs = max(1, int(alpha_rs*num_words))

    # sr
    for _ in range(num_new_per_technique):
        a_words = synonym_replacement(words, n_sr)
        augmented_sentences.append(' '.join(a_words))

    # ri
    for _ in range(num_new_per_technique):
        a_words = random_insertion(words, n_ri)
        augmented_sentences.append(' '.join(a_words))

    # rs
    for _ in range(num_new_per_technique):
        a_words = random_swap(words, n_rs)
        augmented_sentences.append(" ".join(a_words))

    # rd
    for _ in range(num_new_per_technique):
        a_words = random_deletion(words, p_rd)
        augmented_sentences.append(" ".join(a_words))

    augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
    random.shuffle(augmented_sentences)

    if num_aug >= 1:
        augmented_sentences = augmented_sentences[:num_aug]
    else:
        keep_prob = num_aug / len(augmented_sentences)
        augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    augmented_sentences.append(sentence)

    return augmented_sentences

<>:134: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:134: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\user\AppData\Local\Temp/ipykernel_20020/3335745265.py:134: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [42]:
df = pd.read_table('./FairyTale.txt',sep='\n', encoding='cp949', names = ['내용'])
df

,내용
0,"옛날 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 모두 예쁜 왕이 ..."
1,"특히나 막내공주님이 아주 예뻤는데요, 어찌나 아름다운지, 해도 그녀의 얼굴에 햇볕을..."
2,왕의 성은 울창한 숲에 둘러싸여 있었어요.
3,숲에 있는 나이 많은 라임 나무 아래에 우물이 하나 있었어요.
4,"화창한 날에, 막내공주님께서 숲으로 놀러 나와 시원한 분수 옆에 앉았어요."
...,...
9342,게으름뱅이의 사연을 들은 농부는 게으름뱅이를 집으로 돌려보냈습니다. 집으로 돌아가던...
9343,?
9344,"게으름뱅이는 남아있는 옷감을 보며, 산신령이 게으름 피우지 않도록 깨우침을 주셨다고..."
9345,?


In [44]:
df = df[df['내용'] != '?']

In [72]:
df = df.reset_index()
df.drop(['index'], axis = 1, inplace = True)

In [81]:
result = []

In [82]:
for text in df['내용']:
    temp = EDA(text, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9)
    result.append(temp)

In [86]:
df['내용'][0]

'옛날 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 모두 예쁜 왕이 한 분 살고 계셨어요.'

In [84]:
result[0]

['옛날 옛적에 소망을 예쁜 주는 힘이 여전히 통했던 때에, 딸들이 모두 이루어 왕이 한 분 살고 계셨어요.',
 '옛날 옛적에 소망을 이루어 주는 힘이 딸들이 통했던 때에, 여전히 모두 예쁜 왕이 한 분 살고 계셨어요.',
 '계셨어요. 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 모두 예쁜 왕이 한 분 살고 옛날',
 '옛날 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 모두 예쁜 왕이 한 분 살고 계셨어요.',
 '옛날 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 모두 예쁜 왕이 한 분 살고 계셨어요.',
 '옛적에 소망을 힘이 여전히 때에, 딸들이 예쁜 왕이 한 분 살고 계셨어요.',
 '옛날 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 모두 예쁜 왕이 한 분 살고 계셨어요.',
 '옛 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 모두 예쁜 왕이 한 분 살고 계셨어요.',
 '옛날 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 분 모두 예쁜 왕이 한 분 살고 계셨어요.',
 '옛날 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 모두 예쁜 왕이 한 분 살고 계셨어요.']